In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress

# Import API keys
from api_keys import g_key
from api_keys import meteostat_api_key

gmaps.configure(api_key=g_key)

In [10]:
#importing Yellow Cab Taxi Ride data from .csv
# PULocationID = Pickup Location ID
# DOLocationID = Drop-off Location ID

# Contains the info for every taxi cab ride in New York City in January 2019

taxi_data = pd.read_csv("taxi_data/combined_months.csv")

# Contains each official Taxi Pickup Zone in NYC 

pickup_zone_data = pd.read_csv("taxi_data/nyc_pickup_zones.csv")
pickup_zone_data = pickup_zone_data.rename(columns={'OBJECTID': 'PULocationID'})

In [11]:
# Setting up the Pickup Zone Name column
taxi_data['Pickup Zone Name'] = ""

# Merging the 'taxi_data' and 'pickup_zone_data' on PULocationID (aka Zone ID)
# This is essentially grabbing the PULocationID from 'taxi_data' and finding it's name in 'pickup_zone_data'
merged_taxi_zones = pd.merge(taxi_data,pickup_zone_data, how='inner',on="PULocationID")



# Filtering the 'merged_taxi_zones' dataframe so it only include rides that happened in Manhattan
manhattan_taxi_zones = merged_taxi_zones[(merged_taxi_zones['borough']=='Manhattan')]
manhattan_taxi_zones


# counting the number of pickups in each pickup zone
pickup_zone_counts = manhattan_taxi_zones['zone'].value_counts()
#pickup_zone_counts

manhattan_taxi_zones

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Pickup Zone Name,Shape_Leng,zone,borough
0,192660,2,2019-01-02 01:35:53,2019-01-02 01:48:41,1,4.36,1,N,113,236,...,0.5,3.06,0.00,0.3,18.36,NaN,,0.032745,Greenwich Village North,Manhattan
1,2727162,2,2019-01-12 16:49:09,2019-01-12 16:58:14,1,1.11,1,N,113,90,...,0.5,1.66,0.00,0.3,9.96,NaN,,0.032745,Greenwich Village North,Manhattan
2,5756427,2,2019-01-24 19:25:59,2019-01-24 19:39:32,1,1.59,1,N,113,164,...,0.5,2.26,0.00,0.3,13.56,0.0,,0.032745,Greenwich Village North,Manhattan
3,1193505,1,2019-01-06 14:59:36,2019-01-06 15:04:28,1,0.80,1,N,113,90,...,0.5,1.15,0.00,0.3,6.95,NaN,,0.032745,Greenwich Village North,Manhattan
4,4549913,1,2019-01-19 17:49:38,2019-01-19 18:01:26,2,1.20,1,N,113,144,...,0.5,1.95,0.00,0.3,11.75,NaN,,0.032745,Greenwich Village North,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861,2967535,1,2019-02-12 15:36:51,2019-02-12 15:42:28,1,0.80,1,N,116,244,...,0.5,0.00,0.00,0.3,6.30,0.0,,0.068116,Hamilton Heights,Manhattan
5862,7114025,1,2019-03-29 01:46:08,2019-03-29 01:50:11,2,0.90,1,N,116,42,...,0.5,0.00,0.00,0.3,6.80,0.0,,0.068116,Hamilton Heights,Manhattan
5863,7266559,2,2019-05-30 21:48:33,2019-05-30 22:35:24,1,19.74,2,N,116,132,...,0.5,5.00,6.12,0.3,63.92,0.0,,0.068116,Hamilton Heights,Manhattan
5864,5577523,2,2019-06-25 01:14:16,2019-06-25 01:18:50,1,0.73,1,N,116,244,...,0.5,1.58,0.00,0.3,7.88,0.0,,0.068116,Hamilton Heights,Manhattan
